In [2]:
import torch
from policies.policies import DiscterePolicy

In [14]:
p = DiscterePolicy()
ol = torch.randn(512, 1, 64)  # Example input for laser scanner (1 sample, 1 channel, 10 features)
od = torch.randn(512, 4)       # Example input for od (1 sample, 4 features)
og = torch.randn(512, 8)       # Example input for og (1 sample, 4 features)
ov = torch.randn(512, 8) 

In [15]:
p.sample_actions(ol, od, og, ov)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [17]:
params = p.sample_actions(ol, od, og, ov)
params

tensor([[0.1047, 0.0965, 0.1038,  ..., 0.0946, 0.1078, 0.0969],
        [0.1014, 0.1018, 0.1155,  ..., 0.0963, 0.1038, 0.0931],
        [0.1039, 0.1013, 0.1044,  ..., 0.0973, 0.1040, 0.0983],
        ...,
        [0.0955, 0.0987, 0.1093,  ..., 0.0989, 0.1076, 0.1015],
        [0.1018, 0.0929, 0.1061,  ..., 0.0959, 0.1112, 0.0976],
        [0.1012, 0.1057, 0.1074,  ..., 0.0945, 0.1005, 0.0926]],
       grad_fn=<SoftmaxBackward0>)

In [19]:
# params = params.view(512, -1, 10)
max_values = torch.argmax(params, dim=1)
max_values = max_values.squeeze()

In [21]:
act = {
            0: (0, 0),
            1: (1, 0),
            2: (1, 0.25),
            3: (1, -0.25),
            4: (1, 0.5),
            5: (1, -0.5),
            6: (1, 0.75),
            7: (1, -0.75),
            8: (1, 1),
            9: (1, -1),
        }
actions = [act[a.item()] for a in max_values]

In [23]:
actions

[(1, 1),
 (1, 0.25),
 (1, -0.25),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 0),
 (1, 0),
 (0, 0),
 (1, 0.25),
 (1, -0.25),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 0.75),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, -0.25),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 0.75),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, 0.25),
 (1, -0.25),
 (1, 0.25),
 (1, 1),
 (1, -0.25),
 (1, -0.25),
 (1, -0.25),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, -0.25),
 (1, 0.25),
 (1, 1),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 0.25),
 (1, 1),
 (1, 0.25),
 (1, 1),
 (1, -0.25),
 (1, 0.25),
 (1, -0.25),
 (1, -0.25),
 (1, 1),
 (1, 1),
 (1, 0.2

In [24]:
max_values

tensor([8, 2, 3, 2, 2, 8, 1, 1, 0, 2, 3, 2, 8, 2, 6, 8, 8, 2, 8, 8, 0, 2, 8, 8,
        8, 8, 2, 8, 2, 2, 3, 8, 8, 2, 2, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 2, 2, 8,
        2, 6, 8, 2, 2, 8, 2, 8, 8, 2, 8, 8, 2, 2, 2, 3, 2, 8, 3, 3, 3, 2, 2, 8,
        2, 8, 8, 2, 3, 2, 8, 8, 2, 8, 2, 2, 8, 2, 8, 3, 2, 3, 3, 8, 8, 2, 8, 3,
        9, 8, 8, 8, 8, 2, 8, 2, 8, 4, 3, 2, 2, 1, 2, 3, 8, 8, 2, 2, 2, 2, 2, 8,
        8, 3, 3, 8, 3, 2, 3, 8, 8, 2, 2, 2, 2, 8, 8, 1, 3, 3, 2, 2, 2, 2, 0, 8,
        3, 3, 2, 8, 8, 8, 2, 8, 3, 3, 2, 2, 8, 8, 8, 2, 3, 2, 2, 2, 2, 3, 8, 8,
        2, 8, 2, 3, 8, 2, 2, 8, 8, 2, 2, 1, 8, 3, 2, 3, 8, 2, 8, 2, 2, 2, 2, 3,
        8, 8, 2, 2, 2, 2, 8, 8, 2, 2, 2, 2, 8, 1, 2, 3, 8, 3, 2, 2, 2, 3, 8, 3,
        1, 2, 3, 2, 8, 2, 8, 3, 8, 2, 3, 8, 2, 2, 3, 6, 8, 2, 8, 3, 8, 8, 2, 2,
        8, 8, 2, 8, 4, 8, 2, 2, 2, 3, 8, 8, 3, 2, 3, 6, 0, 8, 2, 8, 2, 8, 8, 8,
        8, 8, 2, 2, 2, 8, 2, 8, 6, 2, 8, 2, 2, 2, 2, 1, 1, 8, 2, 6, 2, 8, 2, 8,
        8, 3, 2, 2, 3, 2, 2, 8, 6, 3, 2,